In [2]:
import numpy as np
import random
import os
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from joblib import dump

import itertools
from sklearn.utils import shuffle
from scipy import signal
%matplotlib inline


from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

from sklearn.feature_selection import SelectFdr, chi2

from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score

from utils.svm import preProcess, evaluate
from utils.visualize import showMe
from utils.augment import augment
%load_ext autoreload
%autoreload 2


C:\Users\Nemes\anaconda3\envs\tf\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Nemes\anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\Nemes\anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\Nemes\anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [3]:
input_length = 100
resource_path = 'C:/resources/EMG/Nemes'
post_fix = '_1s_augmented'

train_sessions = os.listdir(resource_path)
train_sessions.pop()                           # Remove last session for testing purpose
test_sessions  = os.listdir(resource_path)

classes = ['Rest', 'Eyebrow','Smile', 'Chew']

records = {}
for c in classes:
    class_data = []
    for session in train_sessions:
        class_data.append(np.load(os.path.join(resource_path,session,c+post_fix+'.npy'),allow_pickle=True))
    records[c] = np.concatenate(class_data)


In [7]:
def showFirstN(data, limit = 5):
    i = 0
    for trial in data:
        showMe(trial)
        if i == 5:
            break
        i += 1


In [ ]:
showFirstN(records['Eyebrow'])

In [4]:
X = np.concatenate((list(records.values())),axis = 0)

s = records['Rest'].shape[0]

y = np.concatenate(( np.zeros(s),np.ones(s),np.ones(s)*2,np.ones(s)*3))


c = list(zip(X, y))
random.seed(42)
random.shuffle(c)
X,y = zip(*c)


X = np.array(X).reshape(-1,6*input_length)
y = np.array(y)
print(X.shape)
print(y.shape)
X, y = augment(X, y)
print(X.shape)
print(y.shape)

(1080, 600)
(1080,)
(2160, 600)
(2160,)


In [5]:
# SMALLER C -> better fit
# HIGHER gamma -> better fit
param_grid = {'C': [1, 10, 100,1000], 'gamma': [1,0.1,0.01,0.001,0.0001]} 

In [7]:
accs = []
models = []
def grid(X_train,y_train, X_test, y_test):
    grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=2)
    grid.fit(X_train,y_train)

    grid_predictions = grid.predict(X_test)
    acc = accuracy_score(y_test,grid_predictions)
    accs.append(acc)
    models.append(grid.best_estimator_)
    if acc > 0.9:
        return True
    else:
        return False


skf = StratifiedKFold(n_splits=10,random_state= 42, shuffle = True)
for train, test in skf.split(X, y):
    
    X_train = X[train]
    y_train = y[train]
    X_test = X[test]
    y_test = y[test]

    if grid(X_train,y_train, X_test, y_test):
        break
    

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] END .......................................C=1, gamma=1; total time=   0.7s
[CV] END .......................................C=1, gamma=1; total time=   0.7s
[CV] END .......................................C=1, gamma=1; total time=   0.7s
[CV] END .......................................C=1, gamma=1; total time=   0.7s
[CV] END .......................................C=1, gamma=1; total time=   0.7s
[CV] END .....................................C=1, gamma=0.1; total time=   0.7s
[CV] END .....................................C=1, gamma=0.1; total time=   0.7s
[CV] END .....................................C=1, gamma=0.1; total time=   0.7s
[CV] END .....................................C=1, gamma=0.1; total time=   0.7s
[CV] END .....................................C=1, gamma=0.1; total time=   0.6s
[CV] END ....................................C=1, gamma=0.01; total time=   0.5s
[CV] END ....................................C=

In [8]:
model = models[accs.index(max(accs))]
for acc in accs:
    print(acc)

0.8981481481481481
0.8935185185185185
0.9212962962962963


In [9]:
for session in test_sessions:
    print("Evaluating session: {}".format(session))
    evaluate(model, session, resource_path, classes, post_fix)

Evaluating session: session_0
Accuracy : 99.0%
[[90  0  0  0]
 [ 0 90  0  0]
 [ 1  0 89  0]
 [ 1  0  0 89]]

Evaluating session: session_1
Accuracy : 99.0%
[[180   0   0   0]
 [  0 180   0   0]
 [  1   1 178   0]
 [  0   0   2 178]]

Evaluating session: session_2
Accuracy : 37.0%
[[32  7 24 12]
 [ 5 50  7 13]
 [18  0 15 42]
 [41  3 18 13]]



In [8]:
dump(model, 'saved_models/svm_03_04.joblib') 

['saved_models/svm_03_04.joblib']